In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import awswrangler as wr

In [2]:
bucket = 'ecommerceanalysis'
dir = 'archive/Oct/'
path = f's3://{bucket}/{dir}'
data = wr.s3.read_csv(path=path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [3]:
data['event_time'] = data['event_time'].str[:19]

In [4]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
data.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [6]:
data.dropna(how = 'any', inplace = True)
data.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [7]:
data.groupby("event_type")["event_type"].count().sort_values(ascending=False)

event_type
view        25201706
cart          809407
purchase      549507
Name: event_type, dtype: int64

In [8]:
#Dropping columns not relevent to our analysis
data = data.drop(columns=['user_id'])
data = data.drop(columns=['user_session'])

In [9]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98
5,2019-10-01 00:00:05,view,1480613,2053013561092866779,computers.desktop,pulser,908.62
8,2019-10-01 00:00:10,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71


In [10]:
#checking data size after null values have been dropped
data.shape

(26560620, 7)

In [11]:
data['event_type'].unique()

array(['view', 'purchase', 'cart'], dtype=object)

In [12]:
#Breaking data into 3 dataframes
view_data = data[data['event_type'] == 'view']
view_data = view_data.drop(columns=['event_type',"category_id"])
view_data.head()

,event_time,product_id,category_code,brand,price
1,2019-10-01 00:00:00,3900821,appliances.environment.water_heater,aqua,33.20
3,2019-10-01 00:00:01,1307067,computers.notebook,lenovo,251.74
4,2019-10-01 00:00:04,1004237,electronics.smartphone,apple,1081.98
5,2019-10-01 00:00:05,1480613,computers.desktop,pulser,908.62
8,2019-10-01 00:00:10,28719074,apparel.shoes.keds,baden,102.71


In [13]:
view_data.shape

(25201706, 5)

In [14]:
#Breaking data into 3 dataframes
purchase_data = data[data['event_type'] == 'purchase']
purchase_data = purchase_data.drop(columns=['event_type',"category_id"])
purchase_data.head()

,event_time,product_id,category_code,brand,price
162,2019-10-01 00:02:14,1004856,electronics.smartphone,samsung,130.76
308,2019-10-01 00:04:37,1002532,electronics.smartphone,apple,642.69
442,2019-10-01 00:07:07,13800054,furniture.bathroom.toilet,santeri,54.42
574,2019-10-01 00:09:26,4804055,electronics.audio.headphone,apple,189.91
603,2019-10-01 00:09:54,4804056,electronics.audio.headphone,apple,161.98


In [15]:
#Checking data size
purchase_data.shape

(549507, 5)

In [16]:
#Breaking data into 3 dataframes
cart_data = data[data['event_type'] == 'cart']
cart_data = cart_data.drop(columns=['event_type',"category_id"])
cart_data.head()

,event_time,product_id,category_code,brand,price
583,2019-10-01 00:09:33,1002524,electronics.smartphone,apple,515.67
680,2019-10-01 00:11:00,4804056,electronics.audio.headphone,apple,161.98
1325,2019-10-01 02:17:59,1004833,electronics.smartphone,samsung,174.76
1654,2019-10-01 02:19:36,1005003,electronics.smartphone,huawei,258.21
1684,2019-10-01 02:19:43,1004750,electronics.smartphone,samsung,197.43


In [17]:
cart_data.shape

(809407, 5)

In [18]:
view_data['category_code'].value_counts()

electronics.smartphone          10598372
computers.notebook               1100127
electronics.clocks               1095290
electronics.video.tv             1054287
electronics.audio.headphone      1011396
                                  ...   
apparel.shorts                       421
apparel.belt                         366
auto.accessories.anti_freeze         296
construction.tools.soldering         201
country_yard.furniture.bench         190
Name: category_code, Length: 126, dtype: int64

In [19]:
purchase_data['category_code'].value_counts()

electronics.smartphone         337575
electronics.audio.headphone     30439
electronics.video.tv            21548
electronics.clocks              16647
appliances.kitchen.washer       16059
                                ...  
apparel.skirt                       3
apparel.jeans                       3
apparel.shorts                      2
apparel.jumper                      2
apparel.jacket                      1
Name: category_code, Length: 120, dtype: int64

In [20]:
cart_data['category_code'].value_counts()

electronics.smartphone         549373
electronics.audio.headphone     51117
electronics.video.tv            36212
appliances.kitchen.washer       21896
electronics.clocks              20270
                                ...  
country_yard.cultivator            11
construction.tools.painting         8
furniture.bedroom.pillow            7
accessories.umbrella                5
apparel.sock                        5
Name: category_code, Length: 89, dtype: int64

In [22]:
# export to csv
# view_data.to_csv('Resources/view_Oct_2019.csv',encoding='utf-8',index=False) # over 2 GB still
purchase_data.to_csv('../e_commerce_analysis_temp/csv/purchase_Oct_2019.csv',encoding='utf-8',index=False)
cart_data.to_csv('Resources/cart_Oct_2019.csv',encoding='utf-8',index=False)